# Developing scripts for data preparation

In [1]:
%load_ext autoreload
%autoreload 2
%load_ext lab_black

In [101]:
from src.data import open_dataset

In [106]:
config = "/g/data/xv83/users/ds0092/active_projects/Squire_2022_CAFE-f6/config/CAFE60v1.atmos.yaml"
test = open_dataset(config)

In [107]:
test

,Array,Chunk
Bytes,564.79 MiB,4.75 MiB
Shape,"(119, 96, 90, 144)","(1, 96, 90, 144)"
Count,2853 Tasks,119 Chunks
Type,float32,numpy.ndarray
,Array,Chunk
Bytes,564.79 MiB,4.75 MiB
Shape,"(119, 96, 90, 144)","(1, 96, 90, 144)"
Count,3573 Tasks,119 Chunks
Type,float32,numpy.ndarray


In [41]:
test = open_dataset("CAFE60v1", "t_ref", "atmos_isobaric_month")

In [ ]:
def make_datasets(datasets, yaml_suffix):
    """ Process dataset according to specifications in a provided set of config files
        and save output ../processed
    """
    ds

In [140]:
def JRA55(realm, variables):
    """Open JRA55 data following specifications in JRA55.yaml"""

    cfg = _load_config("JRA55")

    if isinstance(variables, str):
        variables = [variables]

    if "rename" in cfg:
        variables = _maybe_translate_variables(variables, cfg["rename"])

    if "preprocess" in cfg:
        warnings.warn(
            "preprocess functions were provided but not used because the data does not require concatenation"
        )

    ds = xr.open_dataset(
        f"{cfg['path']}/{realm}.zarr.zip",
        engine="zarr",
        chunks={},
        use_cftime=True,
    )[variables]

    if "rename" in cfg:
        ds = _maybe_rename(ds, cfg["rename"])

    if "normalise" in cfg:
        ds = _normalise(ds, cfg["normalise"])

    if "postprocess" in cfg:
        ds = _composite_function(cfg["postprocess"])(ds)

    return ds


def HadISST(variables):
    """Open HadISST data following specifications in HadISST.yaml"""

    cfg = _load_config("HadISST")

    if isinstance(variables, str):
        variables = [variables]

    if "rename" in cfg:
        variables = _maybe_translate_variables(variables, cfg["rename"])

    if "preprocess" in cfg:
        warnings.warn(
            "preprocess functions were provided but not used because the data does not require concatenation"
        )

    ds = xr.open_dataset(
        f"{cfg['path']}/ocean_month.zarr",
        engine="zarr",
        chunks={},
        use_cftime=True,
    )[variables]
    ds = ds.where(ds > -1000)

    if "rename" in cfg:
        ds = _maybe_rename(ds, cfg["rename"])

    if "normalise" in cfg:
        ds = _normalise(ds, cfg["normalise"])

    if "postprocess" in cfg:
        ds = _composite_function(cfg["postprocess"])(ds)

    return ds


def EN422(variables):
    """Open EN.4.2.2 data following specifications in EN422.yaml"""

    cfg = _load_config("EN422")

    if isinstance(variables, str):
        variables = [variables]

    if "rename" in cfg:
        variables = _maybe_translate_variables(variables, cfg["rename"])

    if "preprocess" in cfg:
        warnings.warn(
            "preprocess functions were provided but not used because the data does not require concatenation"
        )

    ds = xr.open_mfdataset(
        f"{PATHS['EN422']}/*.nc",
        parallel=True,
        use_cftime=True,
    )[variables]

    if "rename" in cfg:
        ds = _maybe_rename(ds, cfg["rename"])

    if "normalise" in cfg:
        ds = _normalise(ds, cfg["normalise"])

    if "postprocess" in cfg:
        ds = _composite_function(cfg["postprocess"])(ds)

    return ds


def CAFEf6(realm, variables):
    """Open CAFEf6 forecast data following specifications in CAFEf6.yaml"""

    cfg = _load_config("CAFEf6")

    if isinstance(variables, str):
        variables = [variables]

    if "rename" in cfg:
        variables = _maybe_translate_variables(variables, cfg["rename"])

    if "preprocess" in cfg:
        preprocess = _composite_function(cfg["preprocess"])
    else:
        preprocess = None

    files = sorted(
        glob.glob(f"{cfg['path']}/c5-d60-pX-f6-????1101/{realm}.zarr.zip")
    )  # Skip May starts

    ds = xr.open_mfdataset(
        files,
        compat="override",
        preprocess=preprocess,
        engine="zarr",
        coords="minimal",
        parallel=True,
    )[variables]

    if "rename" in cfg:
        ds = _maybe_rename(ds, cfg["rename"])

    if "normalise" in cfg:
        ds = _normalise(ds, cfg["normalise"])

    if "postprocess" in cfg:
        ds = _composite_function(cfg["postprocess"])(ds)

    return ds


def CAFEf5(realm, variables):
    """Open CAFE-f5 forecast data following specifications in CAFEf5.yaml"""

    cfg = _load_config("CAFEf5")

    if isinstance(variables, str):
        variables = [variables]

    if "rename" in cfg:
        variables = _maybe_translate_variables(variables, cfg["rename"])

    if "preprocess" in cfg:
        warnings.warn(
            "preprocess functions were provided but not used because the data does not require concatenation"
        )

    ds = xr.open_dataset(
        f"{cfg['path']}/NOV/{realm}.zarr.zip", engine="zarr", chunks={}
    )[variables]

    # Append 2020 forecast from CAFE-f6
    cfg_f6 = _load_config("CAFEf6")

    ds_2020 = xr.open_dataset(
        f"{cfg_f6['path']}/c5-d60-pX-f6-20201101/{realm}.zarr.zip",
        engine="zarr",
        chunks={},
    )[variables]
    ds_2020 = ds_2020.isel(ensemble=range(10))
    ds_2020 = utils.convert_time_to_lead(ds_2020)

    if "rename" in cfg:
        ds = _maybe_rename(ds, cfg["rename"])
        ds_2020 = _maybe_rename(ds_2020, cfg["rename"])

    if "normalise" in cfg:
        ds = _normalise(ds, cfg["normalise"])
        ds_2020 = _normalise(ds_2020, cfg["normalise"])

    if "postprocess" in cfg:
        ds = _composite_function(cfg["postprocess"])(ds)
        ds_2020 = _composite_function(cfg["postprocess"])(ds_2020)

    return xr.concat([ds, ds_2020], dim="init")


def CAFE60v1(realm, variables):
    """Open CAFE60v1 data following specifications in CAFE60v1.yaml"""

    cfg = _load_config("CAFE60v1")

    if isinstance(variables, str):
        variables = [variables]

    if "rename" in cfg:
        variables = _maybe_translate_variables(variables, cfg["rename"])

    if "preprocess" in cfg:
        warnings.warn(
            "preprocess functions were provided but not used because the data does not require concatenation"
        )

    ds = xr.open_dataset(f"{cfg['path']}/{realm}.zarr.zip", engine="zarr", chunks={})[
        variables
    ]

    if "rename" in cfg:
        ds = _maybe_rename(ds, cfg["rename"])

    if "normalise" in cfg:
        ds = _normalise(ds, cfg["normalise"])

    if "postprocess" in cfg:
        ds = _composite_function(cfg["postprocess"])(ds)

    return ds


def CAFE_hist(realm, variables):
    """Open CAFE historical data following specifications in CAFE_hist.yaml"""

    cfg = _load_config("CAFE_hist")

    if isinstance(variables, str):
        variables = [variables]

    if "rename" in cfg:
        variables = _maybe_translate_variables(variables, cfg["rename"])

    if "preprocess" in cfg:
        warnings.warn(
            "preprocess functions were provided but not used because the data does not require concatenation"
        )

    hist = xr.open_dataset(
        f"{cfg['path']}/c5-d60-pX-hist-19601101/ZARR/{realm}.zarr.zip",
        engine="zarr",
        chunks={},
    )[variables]

    ctrl = xr.open_dataset(
        f"{cfg['path']}/c5-d60-pX-ctrl-19601101/ZARR/{realm}.zarr.zip",
        engine="zarr",
        chunks={},
    )[variables].mean("ensemble")

    if "rename" in cfg:
        hist = _maybe_rename(hist, cfg["rename"])
        ctrl = _maybe_rename(ctrl, cfg["rename"])

    if "normalise" in cfg:
        hist = _normalise(hist, cfg["normalise"])
        ctrl = _normalise(ctrl, cfg["normalise"])

    if "postprocess" in cfg:
        hist = _composite_function(cfg["postprocess"])(hist)
        ctrl = _composite_function(cfg["postprocess"])(ctrl)

    drift = ctrl.groupby("time.month").map(lambda x: x - x.mean(["time"]))
    return hist - drift

